# First RUN uncomment

In [ ]:
# !pip install seqeval==0.0.5
# !pip install keras==2.2.4
# !pip install tensorflow==1.14.0
# !git clone https://www.github.com/keras-team/keras-contrib.git
# %cd keras-contrib
# !python setup.py install
# ! pip -q install git+https://www.github.com/keras-team/keras-contrib.git sklearn-crfsuite
# !pip install import_ipynb
# !pip install pyvi
# !pip install ampligraph
# !pip uninstall tensorflow
# !pip install tensorflow==1.13.2

# !pip install seaborn adjustText
# !git clone https://github.com/wyldebeast-wunderliebe/incf.countryutils.git
# !pip install incf.countryutils/.

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# %%capture
# from fastai import *
# from fastai.text import *
import torch
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from bs4 import BeautifulSoup
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from keras.models import Model, Input, load_model
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from keras_contrib.layers import CRF
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/zalo/')
from utils import *
import import_ipynb
from ampligraph.utils import save_model, restore_model
%cd '/content/drive/My Drive/Colab Notebooks/zalo/'
from ner_lstm_crf import *
from scipy.special import expit
from collections import OrderedDict
from tqdm import tqdm

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


/content/drive/My Drive/Colab Notebooks/zalo
importing Jupyter notebook from ner_lstm_crf.ipynb
USE_GPU=False


/usr/local/lib/python3.6/dist-packages/keras_contrib-2.0.8-py3.6.egg/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
/usr/local/lib/python3.6/dist-packages/keras_contrib-2.0.8-py3.6.egg/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy


In [2]:

USE_GPU = torch.cuda.is_available()
print('USE_GPU={}'.format(USE_GPU))
if USE_GPU:
    print('current_device={}'.format(torch.cuda.current_device()))

USE_GPU=False


In [3]:
DATA_PATH = '/content/drive/My Drive/Colab Notebooks/zalo/data/'
MODEL_LSTM_PATH = "/content/drive/My Drive/Colab Notebooks/zalo/model_ner/lstm_crf"
MODEL_KG_PATH = '/content/drive/My Drive/Colab Notebooks/zalo/model_kg/model.pkl'
TEST_DATA_PATH = f'{DATA_PATH}public_test.jsonl'
TRAIN_DATA_PATH = f'{DATA_PATH}train_preprocess.csv'
ENTITY_DATA_PATH = f'{DATA_PATH}entity.csv'
SUBMIT_PATH = f'{DATA_PATH}submit.jsonl'
MAX_LEN = 75

# Load Testset

In [4]:
json_file_test = load_jsonfile(TEST_DATA_PATH)
text_dict = {}
for id in range(len(json_file_test)):
    post_id = json_file_test.iloc[id]['test_id']
    body_content = (json_file_test.iloc[id]['original_doc']['_source']['body'])
    list_cont = []
    for cont in body_content:
        # print(cont['text'])
        if cont['text'] not in list_cont:
            list_cont.append(cont['text'])
    text_dict[post_id] = list_cont

# Load Dict Entity for Pattern Matching

In [5]:
dict_entity = load_entity(ENTITY_DATA_PATH)

# Predict Entity using LSTM-CRF

In [ ]:
# model_lstm= load_model(MODEL_LSTM_PATH,custom_objects={'CRF':CRF, 'crf_loss':crf_loss,'crf_viterbi_accuracy':crf_viterbi_accuracy})

In [6]:
def parse_testset_list(paragraph):
  list_sentence = []
  for index,sent in enumerate(paragraph):
      sub_sent = str(sent).split('.')
      for item in sub_sent:
          list_sentence.append(item)
  return list_sentence

In [ ]:
# dict_entity['team_name']+dict_entity['player_name']

In [7]:
graph = tf.get_default_graph()
def process_one_sent(sentence):
  # 1 lần xử lý 1 câu
  clean_sent = clean_sentence_lstm(sentence).split() 
  embedding_sent=pad_sequences(maxlen=MAX_LEN, sequences=[[word2idx.get(w, 0) for w in clean_sent]], padding="post", value=word2idx["PAD"])
  p = model_lstm.predict(np.array([embedding_sent[0]]))
  p = np.argmax(p, axis=-1)
  pre_entity = {}
  list_label_pre = p[0]
  # print(list_label_pre)
  # combine_entity = dict_entity['team_name'] + dict_entity['player_name']
  # lower_entity = [ent.lower() for ent in combine_entity]
  for w, pred in zip(clean_sent, p[0]):
    if idx2tag[pred].replace(r'PAD','O') != 'O':
        pre_entity[idx2tag[pred]] = w
  # print(list(zip(clean_sent, p[0])))
  return pre_entity

In [8]:
def predict_entity_lstm(paragraph):
  list_node_pred = []
  for sent in paragraph:
    # entity_pred = process_one_sent(sent)
    entity_pred = process_one_sent(sent)
    if len(list(entity_pred.keys())) > 1:
      list_node_pred.append(entity_pred)
  return list_node_pred

## bắt buộc chạy model lstm trước

In [36]:
paragraph = text_dict['21464024']
predict_entity_lstm(paragraph)

[{'score_list_player_name': 'hiếm',
  'score_list_team': 'Mustafi',
  'score_list_time': 'trận'},
 {'score_list_player_name': 'tình', 'score_list_team': 'Matic'},
 {'score_list_team': 'Arsenal', 'score_list_time': '38'},
 {'score_list_player_name': 'thành', 'score_list_time': 'khung'},
 {'score_list_player_name': 'thua', 'score_list_team': 'là'},
 {'score_list_player_name': 'Giroud', 'score_list_team': 'Arsenal'},
 {'score_board_score1': '3',
  'score_board_score2': '4',
  'score_list_player_name': 'Fabregas',
  'score_list_time': '86'},
 {'score_board_score1': '2', 'score_board_score2': '3'}]

# Predicate between entities

In [10]:
# list_subj = ['players_team1','score_board_score1','score_board_score2','score_list_player_name','substitution_list_player_in','substitution_list_player_out','card_list_player_name']
list_subj = ['score_list_player_name','substitution_list_player_in','substitution_list_player_out','card_list_player_name']
list_role = ['isScoreOfTeam','playerScoreAt','playerScoreFor','substituteWith','substituteAt','getCardAt','ofTeamGetCard']
# list_obj = ['players_team1','players_team2','score_list_team','score_list_time','substitution_list_time','card_list_team','card_list_time']
list_obj = ['score_list_team','score_list_time','substitution_list_time','card_list_team','card_list_time']

In [11]:
model_kg = restore_model(model_name_path=MODEL_KG_PATH)
def knowledge_graph(triplet):
  try:
    nparr = np.array(triplet)
    scores = model_kg.predict(nparr)
    probs = expit(scores)
  except:
    pass
    probs = np.array([0],dtype='float32')
  return probs

## Bắt score vs team có thể sai order khắc phục bằng cách đếm số lần ghi score
những trường hợp chỉ có player mà không có team có thể dùng KG để bắt player thuộc về team2 hay team2

In [12]:
def norm_name_team4_player(entity,list_entity):
  dict_match = {}
  dict_match = find_entity_longest_common(entity,list_entity,'default')
  list_entity_appro = []
  if dict_match:
    for match in dict_match: 
      index_entity = match['longest_common_entity_index']
      norm_entity = list_entity[index_entity]
      if norm_entity not in list_entity_appro:
        list_entity_appro.append(norm_entity)
  else:
    list_entity_appro.append(entity)
  return list_entity_appro
  
def norm_name_team(entity,list_entity):
  dict_match = {}
  dict_match = find_entity_longest_common(entity,list_entity,'default')
  if dict_match:
    index_entity = dict_match[0]['longest_common_entity_index']
    norm_entity = list_entity[index_entity]
  else:
    norm_entity = entity
  return norm_entity

In [13]:
def player_score4_team(player_name,team1,team2):
  triplet1 = [player_name,'playerScoreFor',team1]
  triplet2 = [player_name,'playerScoreFor',team2]
  if knowledge_graph(triplet1)[0] > knowledge_graph(triplet2)[0] and knowledge_graph(triplet1)[0] > 0.9:
    return team1
  elif knowledge_graph(triplet2)[0] > knowledge_graph(triplet1)[0] and knowledge_graph(triplet2)[0] > 0.9:
    return team2
  else:
    return None
def confirm_player_score4_team(player,team,dict_entity):
  # list_label_role = [,'ofTeamGetCard']
  # không chuẩn hóa chính xác nhưng cũng gần gần
  list_norm_name = norm_name_team4_player(player,dict_entity['player_name'])
  list_bool = []
  for name in list_norm_name:
  # print(norm_player_name)
    triplet = [name,'playerScoreFor',team]
    prob = knowledge_graph(triplet)[0]
    if prob > 0.75:
      list_bool.append('1')
  if '1' in list_bool:
    return True
def player_getcard_team(player_name,team1,team2):
  triplet1 = [player_name,'ofTeamGetCard',team1]
  triplet2 = [player_name,'ofTeamGetCard',team2]
  if knowledge_graph(triplet1)[0] > knowledge_graph(triplet2)[0] and knowledge_graph(triplet1)[0] > 0.75:
    return team1
  elif knowledge_graph(triplet2)[0] > knowledge_graph(triplet1)[0] and knowledge_graph(triplet1)[0] > 0.75:
    return team2
  else:
    return None
def confirm_player_getcard_team(player,team):
  # list_label_role = [,'ofTeamGetCard']
  triplet = [player,'ofTeamGetCard',team]
  prob = knowledge_graph(triplet)[0]
  if prob > 0.75:
    return True
def catch_score_from_node(list_node_predict):
  list_label_score = ['score_board_score1','score_board_score2']
  score_board = {}
  score_board['score1'] = np.int(0)
  score_board['score2'] = np.int(0)
  for node in list_node_predict:
    for node_key in list(node.keys()):
      if node_key == list_label_score[0] and str(node[node_key]) in [str(num) for num in range(100)]:
        # if int(node[node_key]) >= int(score_board['score1']):
        score_board['score1'] = np.int(node[node_key])
      elif node_key == list_label_score[1] and str(node[node_key]) in [str(num) for num in range(100)]:
        # if int(node[node_key]) >= int(score_board['score2']):
        score_board['score2'] = np.int(node[node_key])
  return score_board
def catch_team_from_node(list_node_predict):
  list_label_team = ['players_team1','players_team2']
  players = {}
  players['team1'] = "None"
  players['team2'] = "None"
  for node in list_node_predict:
    for node_key in list(node.keys()):
      if node_key == list_label_team[0] and node[node_key] != players['team2'] :
        # if int(node[node_key]) >= int(score_board['score1']):
        players['team1'] = node[node_key]
      elif node_key == list_label_team[1] and node[node_key] != players['team1']:
        # if int(node[node_key]) >= int(score_board['score2']):
        players['team2'] = node[node_key]
  return players

def catch_score_list_from_node(list_node_predict,team1,team2):
  list_label_top1 = ['score_list_player_name','score_list_team','score_list_time']
  list_label_top1.sort(reverse=False)

  list_label_top2 = ['score_list_team','score_list_time']
  list_label_top2.sort(reverse=False)

  list_label_top3 = ['score_list_player_name','score_list_time']
  list_label_top3.sort(reverse=False)

  score_list = []
  for node in list_node_predict:
    score_dict_tmp = {}
    list_score_predict = list(node.keys())
    list_score_predict.sort(reverse=False)
    if list_score_predict == list_label_top1:
      # confirm team vs player = KG
      # if confirm_player_score4_team(node['score_list_player_name'],node['score_list_team']):
      # score_dict_tmp['player_name'] = norm_name(node['score_list_player_name'],[dict_entity['player_name']])
      score_dict_tmp['player_name'] = node['score_list_player_name']
      score_dict_tmp['time'] = node['score_list_time']
      score_dict_tmp['team'] = norm_name_team(node['score_list_team'],[team1,team2])
      if score_dict_tmp not in score_list:
        score_list.append(score_dict_tmp)
    elif list_score_predict == list_label_top2:
      score_dict_tmp['player_name'] = "None"
      score_dict_tmp['time'] = node['score_list_time']
      score_dict_tmp['team'] = norm_name_team(node['score_list_team'],[team1,team2])
      if score_dict_tmp not in score_list:
        score_list.append(score_dict_tmp)
    elif list_score_predict == list_label_top3:
      score_dict_tmp['player_name'] = node['score_list_player_name']
      score_dict_tmp['time'] = node['score_list_time']
      score_dict_tmp['team'] = player_score4_team(score_dict_tmp['player_name'],team1,team2)
      if score_dict_tmp not in score_list and score_dict_tmp['team'] != "None":
        score_list.append(score_dict_tmp)
    elif 'score_list_player_name' in list_score_predict:
      score_dict_tmp['player_name'] = node['score_list_player_name']
      score_dict_tmp['team'] = player_score4_team(score_dict_tmp['player_name'],team1,team2)
      score_dict_tmp['time'] = "None"
      if score_dict_tmp not in score_list and score_dict_tmp['team'] != "None" :
        score_list.append(score_dict_tmp)
    if not score_list:
      score_dict_tmp = {}
      score_dict_tmp['player_name'] = "None"
      score_dict_tmp['time'] = "None"
      score_dict_tmp['team'] = "None"
      score_list.append(score_dict_tmp)
  return score_list

def catch_card_list_from_node(list_node_predict,team1,team2):
  list_label_top1 = ['card_list_player_name','card_list_team','card_list_time']
  list_label_top1.sort(reverse=False)

  list_label_top2 = ['card_list_team','card_list_time']
  list_label_top2.sort(reverse=False)

  list_label_top3 = ['card_list_player_name','card_list_time']
  list_label_top3.sort(reverse=False)

  card_list = []
  for node in list_node_predict:
    card_dict_tmp = {}
    list_card_predict = list(node.keys())
    list_card_predict.sort(reverse=False)
    if list_card_predict == list_label_top1:
      if confirm_player_getcard_team(node['card_list_player_name'],node['card_list_team']):
        card_dict_tmp['player_name'] = node['card_list_player_name']
        card_dict_tmp['time'] = node['card_list_time']
        card_dict_tmp['team'] = norm_name_team(node['card_list_team'],[team1,team2])
        if card_dict_tmp not in card_list:
          card_list.append(card_dict_tmp)
    elif list_card_predict == list_label_top2:
      card_dict_tmp['player_name'] = "None"
      card_dict_tmp['time'] = node['card_list_time']
      card_dict_tmp['team'] = norm_name_team(node['card_list_team'],[team1,team2])
      if card_dict_tmp not in card_list:
        card_list.append(card_dict_tmp)
    elif list_card_predict == list_label_top3:
      card_dict_tmp['player_name'] = node['card_list_player_name']
      card_dict_tmp['time'] = node['card_list_time']
      card_dict_tmp['team'] = player_getcard_team(node['card_list_player_name'],team1,team2)
      if card_dict_tmp not in card_list and  card_dict_tmp['team'] != "None":
        card_list.append(card_dict_tmp)
    elif 'card_list_player_name' in list_card_predict:
      card_dict_tmp['player_name'] = node['card_list_player_name']
      card_dict_tmp['time'] = "None"
      card_dict_tmp['team'] = player_getcard_team(node['card_list_player_name'],team1,team2)
      if card_dict_tmp not in card_list and card_dict_tmp['team'] != "None":
        card_list.append(card_dict_tmp)
    if not card_list:
      card_dict_tmp = {}
      card_dict_tmp['player_name'] = "None"
      card_dict_tmp['time'] = "None"
      card_dict_tmp['team'] = "None"
      card_list.append(card_dict_tmp)
  return card_list

In [ ]:
# print(catch_score_from_node(list_node_predict))
# print(catch_team_from_node(list_node_predict))
# print(catch_score_list_from_node(list_node_predict,'Bournemouth','Manchester City'))
# print(catch_card_list_from_node(list_node_predict,'Bournemouth','Manchester City'))

In [ ]:
"""
tăng accuracy bắt team bằng cách kết hợp cả pattern(top3) và lstm_crf
TH xấu: 
  nếu lstm không bắt được thì lấy top2 của pattern
  nếu lstm và pattern cùng bắt ra nhiều hơn 2 team thì dựa vào list_score hoặc list_sub hoặc list_card
  --> tổng team lstm kêt hợp với pattern
"""


'\ntăng accuracy bắt team bằng cách kết hợp cả pattern(top3) và lstm_crf\nTH xấu: \n  nếu lstm không bắt được thì lấy top2 của pattern\n  nếu lstm và pattern cùng bắt ra nhiều hơn 2 team thì dựa vào list_score hoặc list_sub hoặc list_card\n  --> tổng team lstm kêt hợp với pattern\n'

# Combine Pattern + LSTM --> Find Team1/Team2

In [14]:
def catch_teams_pattern(paragraph,dict_entity):
  list_catch_entity = []
  for index,sent in enumerate(paragraph):
    list_sent = str(sent).split('.')
    for item in list_sent:
      # for entity in dict_entity['team_name']:
        # if (item.lower().find(entity.lower()) != -1):
      entity = find_all_entity(item,dict_entity)
      list_catch_entity.append(entity)
  # print(list_catch_entity)
  dict_cnt_team = {}
  list_cnt_team = []
  for ent in list_catch_entity:
    if 'team_name' in list(ent.keys()):
      for team in ent['team_name']:
        # dict_cnt_team[team] = ent['team_name'].count(team)
        list_cnt_team.append(team)
  for team in list_cnt_team:
    dict_cnt_team[team] = list_cnt_team.count(team)
  dict_cnt_top = sorted(dict_cnt_team.items(), key=lambda x: x[1])
# if type(dict_cnt_top) == dict:
  # if len(list(dict_cnt_top.keys())) > 2:
  top3_team = dict_cnt_top[-3:]
  dict_team = {}
  if len(top3_team) > 2:
    dict_team['team1'] = top3_team[2][0]
    dict_team['team2'] = top3_team[1][0]
    dict_team['team3'] = top3_team[0][0]
    return dict_team
  elif len(top3_team) > 1:
    top2_team = dict_cnt_top[-2:]
    dict_team = {}
    dict_team['team1'] = top3_team[1][0]
    dict_team['team2'] = top3_team[0][0]
    return dict_team
  else:
    dict_team = {}
    dict_team['team1'] = "None"
    dict_team['team2'] = "None"
    return dict_team
def major_teams_from_node(list_node_predict):
  dict_team = {}
  combine_team = []
  for node in list_node_predict:
    for key in list(node.keys()):
      if 'team'in key:
        combine_team.append(node[key])
  dict_cnt_teams = {}
  if len(combine_team) > 0:
    for item in combine_team:
      dict_cnt_teams[item] = combine_team.count(item)
    dict_cnt_top = sorted(dict_cnt_teams.items(), key=lambda x: x[1])
    top2_team = dict_cnt_top[-2:]
    if len(top2_team) > 1:
      dict_team = {'team1':top2_team[0][0],'team2':top2_team[1][0]}
  return dict_team


In [15]:
paragraph = text_dict['21488329']
# catch_teams_pattern(['malaysia'],dict_entity)
catch_teams_pattern(paragraph,dict_entity)

{'team1': 'U23 Việt Nam', 'team2': 'U23 Malaysia', 'team3': 'Malaysia'}

In [16]:
 def combine_team_lstm_pattern(paragraph,dict_entity):
  sample_sent = parse_testset_list(paragraph)
  list_node_predict = predict_entity_lstm(sample_sent)
  team_catch_use_lstm = major_teams_from_node(list_node_predict)
  team_catch_use_pattern = catch_teams_pattern(paragraph,dict_entity)
  # print(team_catch_use_pattern)
  list_team_combine = []
  if team_catch_use_pattern:
    for value_pattern in (team_catch_use_pattern.values()):
      if len(team_catch_use_lstm.values()) > 1:
        for value_lstm in (team_catch_use_lstm.values()):
          if len(value_pattern) >= len(value_lstm):
            if value_lstm in value_pattern and value_pattern not in list_team_combine:
              list_team_combine.append(value_pattern)
              if len(list_team_combine) == 2:
                break
          else:
            if value_pattern in value_lstm and value_lstm not in list_team_combine:
              list_team_combine.append(value_lstm)
              if len(list_team_combine) == 2:
                break
      else:
        list_team_combine = list(team_catch_use_pattern.values())[:2]
  elif team_catch_use_lstm and len(team_catch_use_lstm.values()) > 1:
     dict_team_combine = {'team1':team_catch_use_lstm['team1'],'team2':team_catch_use_lstm['team2']}
    # print(list_team_combine)
  if len(list_team_combine) == 2:
    dict_team_combine = {'team1':list_team_combine[0],'team2':list_team_combine[1]}
  elif len(list_team_combine) != 2 and team_catch_use_pattern:
    dict_team_combine = {'team1':team_catch_use_pattern['team1'],'team2':team_catch_use_pattern['team2']}
  else:
    dict_team_combine = {'team1':"None",'team2':"None"}
  return dict_team_combine

In [17]:

def confirm_player_same_team(player1,player2,team1,team2):
  teamofplayer1 = player_score4_team(player1,team1,team2)
  teamofplayer2 = player_score4_team(player2,team1,team2)
  if teamofplayer1 == teamofplayer2:
    return True

def catch_substit_from_node(list_node_predict,team1,team2):
  list_substit = []
  list_label_top1 = ['substitution_list_player_in','substitution_list_time','substitution_list_player_out']
  list_label_top1.sort(reverse=False)

  list_label_top2 = ['substitution_list_time','substitution_list_player_out']
  list_label_top2.sort(reverse=False)

  list_label_top3 = ['substitution_list_player_in','substitution_list_time']
  list_label_top3.sort(reverse=False)
  """
  nếu có nhiều hơn 1 key player trong node: xét same_team
  """
  for node in list_node_predict:
    dict_substit = {}
    list_subs_predict = list(node.keys())
    list_subs_predict.sort(reverse=False)
    # đếm số player xuất hiện trong 1 node
    cnt_player = 0
    list_player = []
    for key_name in list_subs_predict:
      if 'player' in key_name:
        cnt_player += 1
        list_player.append(node[key_name])
    if cnt_player == 2 and list(node.keys())[list(node.values()).index(list_player[0])] != list(node.keys())[list(node.values()).index(list_player[0])]:
      # có 2 player thì check same_team
      if confirm_player_same_team(list_player[0],list_player[1],team1,team2):
        dict_substit['player_in'] = list_player[0]
        dict_substit['player_out'] = list_player[1]
        if 'substitution_list_time' in list(node.keys()):
          dict_substit['time'] = node['substitution_list_time'] 
        elif 'score_list_time' in list(node.keys()):
          dict_substit['time'] = node['score_list_time']
        elif 'card_list_time' in list(node.keys()):
          dict_substit['time'] = node['card_list_time']
        else:
          dict_substit['time'] = "None"
        list_substit.append(dict_substit)
    # phân theo top
    if list_subs_predict == list_label_top1:
        # tmp = {'player_in':,'time':node['substitution_list_time'],''}
      player1 = node['substitution_list_player_in']
      player2 = node['substitution_list_player_out']
      if confirm_player_same_team(player1,player2,team1,team2):
        print("true")
        dict_substit['player_in'] = node['substitution_list_player_in']
        dict_substit['time'] = node['substitution_list_time']
        dict_substit['player_out'] = node['substitution_list_player_out']
        if dict_substit not in list_substit:
          list_substit.append(dict_substit)
    elif list_subs_predict == list_label_top2:
      dict_substit['player_in'] = "None"
      dict_substit['time'] = node['substitution_list_time']
      dict_substit['player_out'] = node['substitution_list_player_out']
      if dict_substit not in list_substit:
        list_substit.append(dict_substit)
    elif list_subs_predict == list_label_top3:
      dict_substit['player_in'] = node['substitution_list_player_in']
      dict_substit['time'] = node['substitution_list_time']
      dict_substit['player_out'] = "None"
      if dict_substit not in list_substit:
        list_substit.append(dict_substit)
    if not list_substit:
      dict_substit = {}
      dict_substit['player_in'] = "None"
      dict_substit['time'] = "None"
      dict_substit['player_out'] = "None"
      list_substit.append(dict_substit)
  return list_substit
  # print(dict_event)

In [ ]:
# # list_node_predict
# catch_substit_from_node(list_node_predict,'U23 Việt Nam','Malaysia')

In [18]:
def confirm_score_board(paragraph,list_node_predict):
  list_score_pattern = []
  dict_score_pattern = {}
  dict_score_lstm = catch_score_from_node(list_node_predict)
  for index,sent in enumerate(paragraph):
    list_sent = str(sent).split('.')
    for item in list_sent:
      score = catch_score(item)
      if score[0] != None:
        list_score_pattern.append(score)
  s1 = np.int(0)
  s2 = np.int(0)
  for item in list_score_pattern:
    if np.int(item[0]) >= s1:
      s1 = np.int(item[0])
    if np.int(item[1]) >= s2:
      s2 = np.int(item[1])
  dict_score_pattern['score1'] = str(s1)
  dict_score_pattern['score2'] = str(s2)
  dict_res = {}
  if dict_score_pattern == dict_score_lstm:
    dict_res = sorted(dict_score_pattern.items(), key=lambda x: x[1])
  else:
    # print(dict_score_lstm)
    dict_res = sorted(dict_score_lstm.items(), key=lambda x: x[1])
  return dict_res

In [ ]:
# print(confirm_score_board(paragraph,list_node_predict))

In [ ]:
# list_node_predict

In [ ]:
# result_tmp = {}
# result_tmp['players'] = combine_team_lstm_pattern(paragraph,dict_entity)
# result_tmp['score_list'] = catch_score_list_from_node(list_node_predict,result_tmp['players']['team1'],result_tmp['players']['team2'])
# result_tmp['card_list'] = catch_card_list_from_node(list_node_predict,result_tmp['players']['team1'],result_tmp['players']['team2'])
# result_tmp['substitution_list'] = confirm_substi_event(list_node_predict,result_tmp['players']['team1'],result_tmp['players']['team2'])
# result_tmp

In [19]:
def confirm_order_team(combine_team_lstm_pattern,catch_score_list_from_node,confirm_score_board):
  list_team = []
  for item in catch_score_list_from_node:
    if item['team'] != "None":
      list_team.append(item['team'])
  team1 = combine_team_lstm_pattern['team1']
  team2 = combine_team_lstm_pattern['team2']
  if team1 in list_team:
    cnt_team1 = list_team.count(team1)
  else:
    cnt_team1 = 0
  if team2 in list_team:
    cnt_team2 = list_team.count(team2)
  else:
    cnt_team2 = 0
  score_big =  confirm_score_board[1][1]
  score_lit =  confirm_score_board[0][1]
  if cnt_team1 >= cnt_team2:
    return {'team1':team1,'team2':team2},{'score1':str(score_big),'score2':str(score_lit)}
  else:
    return {'team1':team1,'team2':team2},{'score1':str(score_lit),'score2':str(score_big)}

In [ ]:
# test_id = '21541461'

In [ ]:
# # paragraph = text_dict['21464024']
# list_result = []
# error_list = []
# for item_id in tqdm(list(text_dict.keys())):
#   try:
#     paragraph = text_dict[item_id]
#     sample_sent = parse_testset_list(paragraph)
#     list_node_predict = predict_entity_lstm(sample_sent)
#     res_final = {}
#     match_summary = {}
#     res_final['0_test_id'] = str(item_id)
#     match_summary['1_players'] = {}
#     match_summary['2_score_board'] = {}
#     match_summary['3_score_list'] = []
#     match_summary['4_card_list'] = []
#     match_summary['5_substitution_list'] = []
#     team_tmp = combine_team_lstm_pattern(paragraph,dict_entity)
#     match_summary['3_score_list'] = catch_score_list_from_node(list_node_predict,team_tmp['team1'],team_tmp['team2'])
#     match_summary['1_players'],match_summary['2_score_board']=confirm_order_team(team_tmp,match_summary['3_score_list'],confirm_score_board(paragraph,list_node_predict))
#     match_summary['4_card_list'] = catch_card_list_from_node(list_node_predict,team_tmp['team1'],team_tmp['team2'])
#     match_summary['5_substitution_list'] = catch_substit_from_node(list_node_predict,team_tmp['team1'],team_tmp['team2'])
#     res_final['match_summary'] = match_summary
#     list_result.append(res_final)
#   except:
#     error_list.append(item_id)
#     print('Failed at: {}'.format(item_id))

In [ ]:
error_list

[]

In [25]:

item_id = '21464024'
paragraph = text_dict[item_id]
sample_sent = parse_testset_list(paragraph)
list_node_predict = predict_entity_lstm(sample_sent)
result_update = {}
result_update['0_test_id'] = str(item_id)
result_update['1_players'] = {}
result_update['2_score_board'] = {}
result_update['3_score_list'] = []
result_update['4_card_list'] = []
result_update['5_substitution_list'] = []
team_tmp = combine_team_lstm_pattern(paragraph,dict_entity)
result_update['3_score_list'] = catch_score_list_from_node(list_node_predict,team_tmp['team1'],team_tmp['team2'])
result_update['1_players'],result_update['2_score_board']=confirm_order_team(team_tmp,result_update['3_score_list'],confirm_score_board(paragraph,list_node_predict))
result_update['4_card_list'] = catch_card_list_from_node(list_node_predict,team_tmp['team1'],team_tmp['team2'])
result_update['5_substitution_list'] = catch_substit_from_node(list_node_predict,team_tmp['team1'],team_tmp['team2'])

In [29]:
result_update['3_score_list']

[{'player_name': 'Alonso', 'team': 'Chelsea', 'time': 'None'},
 {'player_name': 'Costa', 'team': 'Chelsea', 'time': 'None'},
 {'player_name': 'None', 'team': 'Arsenal', 'time': '38'},
 {'player_name': 'Hazard', 'team': 'Chelsea', 'time': '53'},
 {'player_name': 'Fabregas', 'team': 'Arsenal', 'time': 'None'},
 {'player_name': 'Giroud', 'team': 'Arsenal', 'time': 'None'},
 {'player_name': 'Kante', 'team': 'Chelsea', 'time': '8'},
 {'player_name': 'Alonso', 'team': 'Chelsea', 'time': '7'},
 {'player_name': 'Pedro', 'team': 'Chelsea', 'time': '7'},
 {'player_name': 'Fabregas', 'team': 'Arsenal', 'time': '84'},
 {'player_name': 'Fabregas', 'team': 'Arsenal', 'time': '86'},
 {'player_name': 'Gabriel', 'team': None, 'time': '5'},
 {'player_name': 'Giroud', 'team': 'Arsenal', 'time': '6'},
 {'player_name': 'Welbeck', 'team': 'Arsenal', 'time': '6'},
 {'player_name': 'Giroud', 'team': 'Arsenal', 'time': '90'}]

In [26]:
list_node_predict

[{'score_list_player_name': 'Alonso', 'score_list_team': 'Chelsea'},
 {'score_list_player_name': 'Costa', 'score_list_team': 'Chelsea'},
 {'score_list_team': 'Arsenal', 'score_list_time': '38'},
 {'score_list_player_name': 'Hazard', 'score_list_time': '53'},
 {'players_team1': 'Arsenal', 'score_list_time': '85'},
 {'score_list_player_name': 'Fabregas', 'score_list_team': 'Arsenal'},
 {'score_list_player_name': 'Giroud', 'score_list_team': 'Arsenal'},
 {'players_team1': 'Arsenal', 'score_board_score1': '2'},
 {'score_board_score1': '3', 'score_board_score2': '4'},
 {'score_list_player_name': 'Kante', 'score_list_time': '8'},
 {'score_list_player_name': 'Alonso', 'score_list_time': '7'},
 {'score_list_player_name': 'Pedro', 'score_list_time': '7'},
 {'score_list_player_name': 'Fabregas', 'score_list_time': '84'},
 {'score_list_player_name': 'Fabregas', 'score_list_time': '86'},
 {'score_board_score1': '2', 'score_board_score2': '3'},
 {'score_list_player_name': 'Gabriel', 'score_list_tim

In [34]:
player_score4_team('Fabregas','Chelsea','Arsenal')

'Arsenal'

In [ ]:
with open(SUBMIT_PATH,'w') as submit:
  for item in list_result: 
    str_res = str(item).replace(r"'",r'"')
    str_res = str_res.replace(r'0_test_id',r'test_id')
    str_res = str_res.replace(r'1_players',r'players')
    str_res = str_res.replace(r'2_score_board',r'score_board')
    str_res = str_res.replace(r'3_score_list',r'score_list')
    str_res = str_res.replace(r'4_card_list',r'card_list')
    str_res = str_res.replace(r'5_substitution_list',r'substitution_list')
    submit.write(str_res)
    submit.write('\n')
